## Computer Vision and Deep Learning 
## Lab 2 &ndash; Mono Camera Calibration Example
This lab looks into calibrating camera systems. To this end we want to utilise corresponding points within a set of images to help identify intrinsic parameters linking the cameras in the real-world.

There are two ways to approach this lab. You can either grab the calibration board from the front and grab a series of images from the KinectV2 using code similar to the first lab, or you can utilise the data I have captured myself. Both will then be used in the same way later on, but it may be nice to have a bit of personalisation for the data you will be using.

First we need to obtain some images, identify a checkerboard pattern on the RGB image maps and then register the points identifed on the checkerboard across the different images. 

We will utilize OpenCV functionality to undertake a large portion of the pipeline, and then later we will visualise the two x-centric viewpoints we can make from these observations. Camera-centric and Board-centric. 

## Imports <a id="imports"></a>
The following section defines the imports used for the rest of the notebook. 

You shouldn't need to change much here.

In [1]:
# For ndarray handling:
import numpy as np

# For plotting:
import matplotlib.pyplot as plt

# For nice camera plotting colors:
from matplotlib.colors import BASE_COLORS, to_rgb
color_names = list(BASE_COLORS)[:-1] # remove the 'w' color
color_list = [to_rgb(color) for color in color_names]

# For nice smooth colorbar
from matplotlib import cm
cmap = cm.get_cmap('viridis', 6*9) # come back to this later
coloration = np.vstack([elem[:3] for elem in cmap.colors])

# For 3D visualisation:
import open3d as o3d

# For image processing applications and camera calibration:
import cv2

# For reading and writing images:
import imageio

# For Operating System tools and file finding:
import os, glob

ImportError: DLL load failed while importing open3d: The specified module could not be found.

## Load saved images from the disk

This code will load saved images from disk. These could be images that you have downloaded from Canvas, or images saved from a previous session.

We expect a height x width x channel tensor, although a height x width matrix will be expanded to include a singleton channel. 

For the provided images the sizes are: 
* Colour image size: 1080 (height) by 1920 (width) by 3 (channel, RGB)

In [2]:
# Get list of filenames for photos with checkerboard pattern.
path_to_data = os.getcwd() + '\CVDL_Lab2_Data' # Edit this if data is another directory.
filenames = glob.glob(path_to_data + '\*.png')
n_files = len(filenames)

# Load the files from disk into np ndarrays.
images = []
for i_file in range(n_files):
    images.append(imageio.imread(filenames[i_file]))
    print(f'Loaded image {i_file} of {n_files}')

# Check first image shape, and ensure that it is a rank 3 tensor.
if len(images[0].shape) == 2:
    images = [np.expand_dims(image, -1) for image in images]

# Create some shape constants for use later.
IMG_H, IMG_W, IMG_C = images[0].shape
print(f'Image shapes are: {images[0].shape}')

NameError: name 'os' is not defined

## Plot frames
In this following section we want to plot the images we have taken. This should only include images of checkerboard calibration patterns.

In [ ]:
# Create a new figure and calculate how many rows and columns are to be plotted.
fig = plt.figure(figsize=(20, 20))
n_subplot_cols = 3
n_subplot_rows = np.ceil(n_files/n_subplot_cols).astype(np.int)

# Loop over our captured frames, and subplot each image.
for i_file in range(n_files):
    plt.subplot(n_subplot_rows, n_subplot_cols, i_file+1)
    plt.imshow(images[i_file])
    plt.axis('off')

## Detect the checkerboard pattern in an image
Using OpenCV, we can use the `findChessboardCorners()` method to identify a standard checkerboard pattern, commonly by identification of Harris corners. This method takes in the orignal image in grayscale, which we use `cvtColor` to achieve, and a tuple of the internal corner counts `checkerboard_pattern`.

The method takes in the following: 
* image - An image potentially containing a chessboard pattern.
* patternSize - A tuple describing the number of inner corners to be detected in the image, given by (points_per_row, points_per_column).

Further detail on this method can be found at https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga93efa9b0aa890de240ca32b11253dd4a.

In [ ]:
#  TODO: Provide the correct checkerboard corner pattern tuple, as described above.
checkerboard_pattern =  [8,8]

# Detect checkerboard in each image
image_corners = []

for i_file in range(n_files):
    
    # Grayscale the image if needed.
    image = images[i_file]
    if IMG_C == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    
    # Detect the corners in the image.
    pattern_found, image_corner = cv2.findChessboardCorners(image, checkerboard_pattern) # Locate pattern in image
    
    if pattern_found:
        # Likely don't need to refine, but if needed then uncomment these two lines.
        #subpixel_criteria =  (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001) # Stopping criteria for refinement
        #img_corner = cv2.cornerSubPix(image, img_corner, (11, 11), (-1, -1), subpixel_criteria)
        
        # Add detected corners into list.
        image_corners.append(image_corner[::-1])
        
        # Vizualise the detected patterns.
        plt.figure(figsize=(20, 20))
        plt.imshow(image, cmap='gray')
        corners = np.squeeze(image_corners[-1])
        plt.scatter(corners[:, 0], corners[:, 1], s=50, c=coloration)
        
        # Add annotations to the image for clarity.
        for i_corner in range(corners.shape[0]):
            plt.text(corners[i_corner,0], corners[i_corner, 1], f'{i_corner}')
        plt.axis('off')

n_successes = len(image_corners)
print('Number of successfully found patterns is: {0} out of {1}'.format(n_successes, n_files))

## Calibrate camera using detected patterns
The following cell looks to calibrate the camera using using the identified corners from the patterns on the calibration board in each image.

First we build a coordinate system for the pattern, this requires specifying the 3D points of the corners on the board from the board's world coordinate system. For example: if we have a board of size (3, 2) we would specify corners at: 
    [[0, 0], [0, 1], [0, 2], 
     [1, 0], [1, 1], [1, 2]]

Next we use OpenCV functionality to calibrate the camera with the `calibrateCamera()` function. This returns the intrinsic camera matrix, the distortion coefficients for our camera, and the rotation and translation vectors for each board. API for `calibrateCamera()`: https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga687a1ab946686f0d85ae0363b5af1d7b Your job here is to figure out which of the returned arguments is the intrinsic matrix.

We then create a dictionary, `camera_extrinsics`, containing the extrinsic parameters for each set of points found. This contains the rotation matrix and translation vector. Note here that we turn the rotation vectors `r_vec` into the rotation matrix using the `Rodrigues()` function.

In [ ]:
# Create the coordinate points of our checkerboard pattern (meshgrid makes this easier)
SQUARE_H = 1 # Height of the squares on the board.
SQUARE_W = SQUARE_H
board = np.zeros((np.product(checkerboard_pattern), 3), dtype=np.float32)
row_points, col_points = np.meshgrid(np.arange(0, SQUARE_H * checkerboard_pattern[0], SQUARE_H),
                                     np.arange(0, SQUARE_W * checkerboard_pattern[1], SQUARE_W))

# Create board by stacking the coordinate mesh.
board = np.stack([row_points.flatten(), 
                  col_points.flatten(),
                  np.zeros_like(row_points.flatten())], axis=-1)

# Cast to float32 for calibration.
board = board.astype(np.float32)

# Create a list of the checkerboard patterns by duplicating it for each successfully detected chessboard.
boards = [board for i in range(n_successes)]

# Calibrate camera.
retval, cameraMatrix, distortion, r_vec, t_vec = cv2.calibrateCamera(boards, image_corners, [IMG_W, IMG_H], None, None)

# Store camera intrinsic matrix.
# TODO: Identify the camera's intrinsic matrix from the list of variables returned by the calibrateCamera method above.
camera_intrinsic = []

# Store camera extrinsic matrices.
camera_extrinsics = {'rotation_matrix' : [],
                     'translation_matrix' : []}

for i_board in range(n_successes):
    camera_extrinsics['rotation_matrix'].append(np.asarray(cv2.Rodrigues(r_vec[i_board])[0]))
    camera_extrinsics['translation_matrix'].append(np.asarray(t_vec[i_board]))

# Translate boards to world coordinate system and plot camera-centric viewpoint
In this cell we will transform the board coordinate system into the world coordinate system of the camera. We can then plot these points in the 3D space relative to our camera centred at (0, 0, 0). This makes the assumption that our camera is fixed and we move our checkerboard pattern around.

We loop over our identified board patterns and translate them into the camera's world coordinate system via the rotation and translation identified via calibration.

Your job here is apply the rotation and translation of the board's coordinates into the world coordinate system. Open3D's PointCloud object has two methods which are useful. Be sure to think about where the origin may be. You will also need to visualise the camera within the scene, this requires knowing the size of the image and the camera intrinsics (sound familiar?).

In [ ]:
# Create base board pointcloud object, this will then be duplicated and placed into our camera coordinate system.
board_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(board))

# Add each of the boards to their position in the scene.
pcd = []
for i_board in range(n_successes):
    
    # Obtain the rotation matrices and translation vectors for each board.
    R = camera_extrinsics['rotation_matrix'][i_board]
    T = camera_extrinsics['translation_matrix'][i_board]
    
    # Move board coordinates to camera coordinate system.
    # TODO: Apply the necessary projection of the board into the scene.
    #       Open3D has two ways to do this: one uses two method calls, the other uses only one.
    #       Either way you use is okay. 
    #       Slide 8 will help you here. As will the Open3D PointCloud API at:
    #       http://www.open3d.org/docs/release/python_api/open3d.geometry.PointCloud.html
    i_board_pcd = o3d.geometry.PointCloud(board_pcd)
    i_board_pcd = 'TODO' # Move the points into the camera coordinate system. 
    
    # Add the ith board to the scene and give it a color.
    pcd.append(i_board_pcd)
    pcd[-1].paint_uniform_color(color_list[i_board % (len(color_list))])
    
    # Add the board's coordinate origin to the scene (in the correct place).
    # TODO: Apply the necessary projection of the board's origin into the scene.
    #       This will be very similar to the above.
    board_origin = o3d.geometry.TriangleMesh.create_coordinate_frame()
    board_origin = 'TODO'
    pcd.append(board_origin)
    
# Add a visual representation of our camera into the scene.
# TODO: Here you will need to identify the required parts of the intrinsic matrix.
#       Open3D won't let you just put the 3x3 matrix in, you will need to pass them in by parameter.
#       Slide 13 should help you here.
o3d_cam_intrinsic = o3d.camera.PinholeCameraIntrinsic(width='TODO',
                                                      height='TODO',
                                                      fx='TODO',
                                                      fy='TODO',
                                                      cx='TODO',
                                                      cy='TODO')

# Add the camera and it's coordinate origin to the scene. 
pcd.append(o3d.geometry.LineSet.create_camera_visualization(o3d_cam_intrinsic, np.eye(4), scale=1))
pcd.append(o3d.geometry.TriangleMesh.create_coordinate_frame())

# Visualize the PointCloud.
o3d.visualization.draw_geometries(pcd,
                                  lookat=np.asarray([ 0.0, 0.0, 0.0]),
                                  up=np.asarray([0.0, -1.0, 0.0]),
                                  front=np.asarray([0.0, 0.0, -1.0]),
                                  zoom=1)

# Translate cameras and plot the pattern-centric viewpoints
In this cell we will transform the camera coordinate system into the world coordinate system of the board. We can then plot these points in the 3D space relative to our board centred at (0, 0, 0). This makes the assumption that our pattern is fixed and we move our camera around.

We loop over each camera viewpoint in our identified board patterns and translate them into the board's world coordinate system via the inversed of the translation and rotation identified via calibration.

In [ ]:
# Add our single board to the scene. Give it a smooth color so we can see which way up it is.
pcd = []
board_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(board))
board_pcd.colors = o3d.utility.Vector3dVector(coloration)
pcd.append(board_pcd)

# TODO: Here you will need to identify the required parts of the intrinsic matrix.
#       This is the same as in the previous cell.
o3d_cam_intrinsic = o3d.camera.PinholeCameraIntrinsic(width='TODO',
                                                      height='TODO',
                                                      fx='TODO',
                                                      fy='TODO',
                                                      cx='TODO',
                                                      cy='TODO')

# Add the coordinate origin of the board to the scene. 
pcd.append(o3d.geometry.TriangleMesh.create_coordinate_frame())

for i_camera in range(n_successes):
    
    # Obtain the rotation matrices and translation vectors for each camera.
    R = camera_extrinsics['rotation_matrix'][i_camera]
    T = camera_extrinsics['translation_matrix'][i_camera]
    
    # Open3D requires a 4x4 camera extrinsic matrix, rather than the expected 3x4. Why?
    cam_extrinsic = np.vstack([np.hstack([R, T]),
                               np.asarray([0, 0, 0, 1])])
    
    # Create a visualisation of the ith camera in the scene, with a nice color.
    pcd.append(o3d.geometry.LineSet.create_camera_visualization(o3d_cam_intrinsic, cam_extrinsic, scale=1))
    pcd[-1].paint_uniform_color(color_list[i_camera % (len(color_list))])
    
    # Add the camera coordinate origin to the scene. 
    camera_origin = o3d.geometry.TriangleMesh.create_coordinate_frame()
    camera_origin = camera_origin.translate(-T)
    camera_origin = camera_origin.rotate(R.T, [0,0,0])
    pcd.append(camera_origin)
    

# Visualize the PointCloud.
o3d.visualization.draw_geometries(pcd,
                                  lookat=np.asarray([ 0.0, 0.0, 0.0]),
                                  up=np.asarray([0.0, -1.0, 0.0]),
                                  front=np.asarray([0.0, 0.0, -1.0]),
                                  zoom=2)